# PART 7: SpaceX Data Analysis with Plotly & Dash  

Importing required libraries 

In [2]:
# importing dash after installing dash using the system terminal (pip install dash)
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

Loading the data into Pandas dataframe

In [10]:
import requests
from io import BytesIO

URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Fetch data from the URL using requests
response = requests.get(URL)

# Read the content of the response into a pandas DataFrame
if response.status_code == 200:
    content = BytesIO(response.content)
    spacex_df = pd.read_csv(content)
else:
    print("Failed to fetch the data.")

df.head(5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [19]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Define or load spacex_df here
# spacex_df = ...

# Initializing the Dash app
app = dash.Dash(__name__)

# Create dropdown options and year list
dropdown_options = [
    {'label': 'All sites', 'value': 'ALL'},
    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
]

# Defining the layout of the dashboard
app.layout = html.Div([
    html.H1("SpaceX Launch Records Dashboard", style={'textAlign': 'center'}),
    html.Div([
        html.Label("Select Site:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='ALL',
            placeholder='Select site',
            searchable=True
        )
    ]),
    html.Div(dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={0: '0',
               100: '100'},
        value=[0, 10000]
    )),
    dcc.Graph(id='success-pie-chart'),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Function decorator to specify function input and output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='dropdown-statistics', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(
            spacex_df,
            names='class',
            title='Total Success Launches',
            labels={'1': 'Success', '0': 'Failure'}
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_count = filtered_df[filtered_df['class'] == 1].shape[0]
        failure_count = filtered_df[filtered_df['class'] == 0].shape[0]
        
        fig = px.pie(
            names=['Success', 'Failure'],
            values=[success_count, failure_count],
            title=f'Success-Failure Distribution for {entered_site}',
            labels={'Success': 'Success', 'Failure': 'Failure'}
        )
        return fig
    pass

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='dropdown-statistics', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)

def update_payload_scatter(entered_site, payload_range):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Payload Mass vs Launch Outcome (All Sites)',
            labels={'class': 'Launch Outcome'}
        )
    else:
        filtered_df = spacex_df[(spacex_df['Launch Site'] == entered_site) &
                                (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f'Payload Mass vs Launch Outcome ({entered_site})',
            labels={'class': 'Launch Outcome'}
        )
    
    return fig
pass 

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)